# Metrics, validation strategies and baselines

В данном jupyter notebook рассматриваются примеры того, какие схемы валидации и метрики используются в рекомендательных системах.
Также построим простые модели (бейзлайны) на данных МТС Библиотеки. 

* [Preprocessing](#preprocessing)
* [General remarks](#general-remarks)
* [Metrics](#metrics)
    * [Regression](#regression)
    * [Classification](#classification)
    * [Ranking](#ranking)
* [Validation strategies](#validation)
* [Baselines](#baselines)

In [1]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise



<a id="preprocessing"></a>
# Preprocessing

Загрузим наши данные, теперь уже с фичами, и применим знания из [pandas-scipy-for-recsys](https://www.kaggle.com/sharthz23/pandas-scipy-for-recsys)

In [2]:
df = pd.read_csv('./interactions.csv')
df_users = pd.read_csv('./users.csv')
df_items = pd.read_csv('./items.csv')

## Interactions

In [3]:
df.head(1)

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01


In [4]:
df_items.head(1)

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886


In [5]:
df_items.head(1)

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886


In [6]:
df.head(10)

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01
5,63454,78434,87,NaN,2018-01-01
6,127451,14876,69,NaN,2018-01-01
7,42797,315927,69,5.0,2018-01-01
8,47287,258483,22,NaN,2018-01-01
9,23439,9762,74,4.0,2018-01-01


In [7]:
df['start_date'] = pd.to_datetime(df['start_date'])

In [8]:
duplicates = df.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = df[duplicates].sort_values(by=['user_id', 'start_date'])
df = df[~duplicates]

In [9]:
df_duplicates

,user_id,item_id,progress,rating,start_date
235670,523,49329,50,NaN,2018-04-24
322043,523,49329,91,NaN,2018-06-05
538989,2758,113728,56,NaN,2018-09-16
894709,2758,113728,10,NaN,2019-03-04
1157841,4804,114245,99,NaN,2019-07-06
...,...,...,...,...,...
1486973,154892,298192,4,NaN,2019-12-10
478655,156948,38118,78,NaN,2018-08-19
1053656,156948,38118,36,5.0,2019-05-18
1115537,158041,208145,19,NaN,2019-06-16


In [10]:
df

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01
...,...,...,...,...,...
1533073,76968,285394,95,NaN,2019-12-31
1533074,153877,285394,76,5.0,2019-12-31
1533075,90021,73789,97,NaN,2019-12-31
1533076,6452,77993,39,NaN,2019-12-31


In [11]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
df = df.append(df_duplicates.reset_index(), ignore_index=True)

C:\Temp\ipykernel_3680\1901207371.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_duplicates.reset_index(), ignore_index=True)


In [12]:
df['progress'] = df['progress'].astype(np.int8)
df['rating'] = df['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532998 entries, 0 to 1532997
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1532998 non-null  int64               
 1   item_id     1532998 non-null  int64               
 2   progress    1532998 non-null  int8                
 3   rating      285355 non-null   Sparse[float32, nan]
 4   start_date  1532998 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 38.7 MB


In [14]:
df.to_pickle('interactions_preprocessed.pickle')

## Users

In [15]:
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [16]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142888 entries, 0 to 142887
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  142888 non-null  int64  
 1   age      142742 non-null  object 
 2   sex      136626 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.3+ MB


In [17]:
df_users.nunique()

user_id    142888
age             6
sex             2
dtype: int64

In [18]:
df_users['age'] = df_users['age'].astype('category')
df_users['sex'] = df_users['sex'].astype(pd.SparseDtype(np.float32, np.nan))

In [19]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142888 entries, 0 to 142887
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype               
---  ------   --------------   -----               
 0   user_id  142888 non-null  int64               
 1   age      142742 non-null  category            
 2   sex      136626 non-null  Sparse[float32, nan]
dtypes: Sparse[float32, nan](1), category(1), int64(1)
memory usage: 2.3 MB


In [20]:
interaction_users = df['user_id'].unique()

In [21]:
common_users = len(np.intersect1d(interaction_users, df_users['user_id']))

In [22]:
common_users

135677

In [23]:



users_only_in_interaction = len(np.setdiff1d(interaction_users, df_users['user_id']))
users_only_features = len(np.setdiff1d(df_users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')

Кол-во пользователей - 158811
Кол-во пользователей c взаимодействиями и фичами - 135677 (85.43%)
Кол-во пользователей только c взаимодействиями - 15923 (10.03%)
Кол-во пользователей только c фичами - 7211 (4.54%)


In [24]:
df_users.to_pickle('users_preprocessed.pickle')

## Items

In [25]:
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [26]:
df_items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59599 entries, 0 to 59598
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       59599 non-null  int64 
 1   title    59599 non-null  object
 2   genres   59568 non-null  object
 3   authors  52714 non-null  object
 4   year     46720 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.3+ MB


In [27]:
def num_bytes_format(num_bytes, float_prec=4):
    units = ['bytes', 'Kb', 'Mb', 'Gb', 'Tb', 'Pb', 'Eb']
    for unit in units[:-1]:
        if abs(num_bytes) < 1000:
            return f'{num_bytes:.{float_prec}f} {unit}'
        num_bytes /= 1000
    return f'{num_bytes:.4f} {units[-1]}'

In [28]:
num_bytes = df_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes)

'28.1660 Mb'

In [29]:
df_items.nunique()

id         59599
title      57358
genres     10769
authors    17265
year        1053
dtype: int64

Почему колонка `year` типа `object`, а не `int`?

In [30]:
df_items['year'].value_counts().tail(25)

1954, 1955, 1956                1
1962,1970                       1
1998-2018                       1
1906-1913                       1
1928, 1930                      1
1971, 1989                      1
1799                            1
1865, 1871                      1
2002, 2004                      1
1915, 1923, 1934                1
1924, 1932, 1934                1
1609                            1
1954, 1964                      1
1945, 1938-1949                 1
1857, 1878, 1908, 1916,         1
1942, 1942, 1958                1
1912, 1918, 1919                1
1812-1819                       1
1945, 1949                      1
1947, 1957, 1958, 1969, 2005    1
1957, 1966, 1970                1
1932, 1976                      1
1987, 1989                      1
1929, 1931                      1
1965,1966,1967,1968             1
Name: year, dtype: int64

In [31]:
df_items[df_items['year'] == '1898, 1897, 1901']

,id,title,genres,authors,year
44131,273885,"«Мальчик, который рисовал кошек» и другие исто...","Ужасы,Мистика,Зарубежная классика,Литература 1...",Лафкадио Хирн,"1898, 1897, 1901"


In [32]:
for col in ['genres', 'authors', 'year']:
    df_items[col] = df_items[col].astype('category')

In [33]:
df_items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59599 entries, 0 to 59598
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       59599 non-null  int64   
 1   title    59599 non-null  object  
 2   genres   59568 non-null  category
 3   authors  52714 non-null  category
 4   year     46720 non-null  category
dtypes: category(3), int64(1), object(1)
memory usage: 2.3+ MB


In [34]:
num_bytes = df_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes)

'17.7194 Mb'

In [35]:
num_bytes

17719446

In [36]:
df_items

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873
...,...,...,...,...,...
59594,45640,МК Московский Комсомолец 291-2019,"Политология,Книги по экономике,Газеты",NaN,2019
59595,321616,МК Московский Комсомолец 292-2019,"Политология,Книги по экономике,Газеты",NaN,2019
59596,125582,Известия 248-249-2019,"Политология,Общая история,Газеты",NaN,2019
59597,33188,Men's Health 01-2020,Журнальные издания,NaN,2019


In [37]:
df_items['year'].unique()

['1886', '1894', '1869', '1831', '1873', ..., '2006-2014', '2006-2019', '2004, 2007', '2014–2019', '1965,1966,1967,1968']
Length: 1054
Categories (1053, object): ['04.01.2014', '04.08.2010', '1000', '1048–1123', ..., 'ок. 355 г. до н. э.', 'около 1900 г.', 'около 500 г. до н.э.', 'первая половина XIX dtrf']

In [38]:
interaction_items = df['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, df_items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, df_items['id']))
items_only_features = len(np.setdiff1d(df_items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 59599
Кол-во книг c взаимодействиями и фичами - 59599 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [39]:
df_items.to_pickle('items_preprocessed.pickle')